## Hands-on
### Quantifying and Reducing Gender Stereotypes in Portuguese Word Embeddings

Based in the following paper: Man is to Computer Programmer as Woman is to Homemaker? Debiasing Word Embeddings by Tolga Bolukbasi, Kai-Wei Chang, James Zou, Venkatesh Saligrama, and Adam Kalai. Proceedings of NIPS 2016.

In [3]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions_pt

### Part1 
#### Gender Bias in Word Embedding


- #### Step 1: Load data

In [4]:
# load google news word2vec
E = WordEmbedding('./embeddings/cbow_s300.txt')


*** Reading data from ./embeddings/cbow_s300.txt
(934966, 300)
934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia
934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia


In [17]:
# load professions
professions = load_professions_pt()
profession_words = [p[0] for p in professions]

Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


- #### Step 2: Define Gender Direction

In [19]:
# gender direction
v_gender = E.diff('ela', 'ele')

- #### Step 3: Generating analogies of "Homem: x :: Mulher : y"

In [6]:
"""a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)"""

'a_gender = E.best_analogies_dist_thresh(v_gender)\n\nfor (a,b,c) in a_gender:\n    print(a+"-"+b)'

In [10]:
professions_pair=[]

with open('./data/professions_analogies_pt.txt', "r") as f:
    for line in f:
        line=line.replace("\n","").replace("(","").replace(")","").replace("'","").replace(" ","")
        male=line.split(",")[1]
        fema=line.split(",")[0]
        professions_pair.append([male,fema])

In [11]:
def evaluate_w2c(E,professions_pair):
       
   #def evaluete_tuples(E,professions_pair):
    tmp_file='./data/temp.bin'
    E.save_w2v(tmp_file)
    print ("saved")
    from gensim.models import KeyedVectors
    tmp_file='./data/temp.bin'
    word_vectors = KeyedVectors.load_word2vec_format(tmp_file, binary=True)
    with open("./data/professions_analogies_pt.txt", "r") as ins:
        for line in ins:
            line=line.replace("\n","").replace("(","").replace(")","").replace("'","").replace(" ","")
            male=line.split(",")[1]
            fema=line.split(",")[0]
            professions_pair.append([male,fema])

    len_= len(professions_pair)
    acc = 0
    for p in professions_pair:
        male=p[0]
        fema=p[1]
        #word_vectors.
        if (male in word_vectors.vocab and fema in word_vectors.vocab):
            x = word_vectors.most_similar(positive=['mulher', male], negative=['homem'])[0][0]
            print (male+"="+x+"-"+fema)
            if x== fema:
                acc+=1

    out = acc/float(len_)
    print (acc)
    print (len_)
    print ("accuracy =",out)
    return out



evaluate_w2c(E,professions_pair)

saved
advogado=empresária-advogada
arquiteto=empresária-arquiteta
ator=empresária-atriz
bibliotecário=aluna-bibliotecária
biólogo=empresária-bióloga
blogueiro=empresária-blogueira
bombeiro=empresária-bombeira
cabeleireiro=empresária-cabeleireira
camareiro=dama-camareira
cantor=empresária-cantora
consultor=empresária-consultora
contador=tia-contadora
coordenador=empresária-coordenadora
coreógrafo=empresária-coreógrafa
costureiro=empresária-costureira
cozinheiro=empresária-cozinheira
dançarino=empresária-dançarina
decorador=empresária-decoradora
diagramador=empresária-diagramadora
diretor=empresária-diretora
empresário=empresária-empresária
enfermeiro=empresária-enfermeira
engenheiro=empresária-engenheira
escritor=empresária-escritora
estagiário=aluna-estagiária
fotógrafo=empresária-fotógrafa
garçom=garçonete-garçonete
historiador=empresária-historiadora
instrutor=aluna-instrutora
juíz=padre-juíza
matemático=empresária-matemática
médico=empresária-médica
pedagogo=empresária-pedagoga
pesq

0.061224489795918366

- #### Step 4: Analyzing gender bias in word vectors associated with professions

In [8]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.09879555, 'ajudante'),
  (-0.05029242, 'política'),
  (-0.024776038, 'jornalismo'),
  (-0.018601764, 'design'),
  (-0.014257116, 'enfermagem'),
  (-0.013866575, 'direito'),
  (-0.012364276, 'motorista'),
  (0.0020164233, 'policial'),
  (0.026986431, 'arquitetura'),
  (0.033040524, 'medicina'),
  (0.03306193, 'antropologia'),
  (0.034641955, 'artes'),
  (0.037827827, 'computação'),
  (0.04257617, 'arqueologia'),
  (0.053356074, 'contabilidade'),
  (0.05761321, 'psicologia'),
  (0.078031376, 'engenharia')],
 [(-0.09879555, 'ajudante'),
  (-0.05029242, 'política'),
  (-0.024776038, 'jornalismo'),
  (-0.018601764, 'design'),
  (-0.014257116, 'enfermagem'),
  (-0.013866575, 'direito'),
  (-0.012364276, 'motorista'),
  (0.0020164233, 'policial'),
  (0.026986431, 'arquitetura'),
  (0.033040524, 'medicina'),
  (0.03306193, 'antropologia'),
  (0.034641955, 'artes'),
  (0.037827827, 'computação'),
  (0.04257617, 'arqueologia'),
  (0.053356074, 'contabilidade'),
  (0.05761321, 'psicologia')

### Part 2
#### Debias

- #### Step 5:

In [5]:
from debiaswe.debias import debias

In [6]:
with open('./data/definitional_pairs_pt.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs_pt.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed_pt.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['mulher', 'homem'], ['garota', 'garoto'], ['ela', 'ele'], ['mãe', 'pai'], ['filha', 'filho'], ['moça', 'moço'], ['fêmea', 'macho'], ['dela', 'dele'], ['própria', 'próprio']]
gender specific 158 ['atriz', 'atrizes', 'tia', 'tias', 'bacharel', 'bacharela', 'bailarina', 'barbearia', 'barítono', 'barba']


In [24]:
debias(E, gender_specific_words, defs, equalize_pairs)

sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

male, female = sp_debiased[0:20], sp_debiased[-20:]
debiased_pairs = []
for i, j in zip(male, female):
    debiased_pairs.append([i[1],j[1]])

evaluate_w2c(E,debiased_pairs)

934966 words of dimension 300 : </s>, ,, de, ., ..., indispô-la, microquimerismo, matro, sentar-se-ia
{('PADRE', 'FREIRA'), ('PRÓPRIO', 'PRÓPRIA'), ('potro', 'potranca'), ('TIO', 'TIA'), ('HOMEM', 'MULHER'), ('Príncipe', 'Princesa'), ('Homem', 'Mulher'), ('Dele', 'Dela'), ('DELE', 'DELA'), ('príncipe', 'princesa'), ('IRMÃOS', 'IRMÃS'), ('testosterona', 'estrogênio'), ('filhos', 'filhas'), ('Irmão', 'Irmã'), ('IRMÃO', 'IRMÃ'), ('homem', 'mulher'), ('aluno', 'aluna'), ('Menino', 'Menina'), ('VÔ', 'VÓ'), ('Reis', 'Rainhas'), ('EX NAMORADA', 'EX NAMORADO'), ('próprio', 'própria'), ('menino', 'menina'), ('Vereador', 'Vereadora'), ('pais', 'mães'), ('Cavalheiros', 'Damas'), ('Ele', 'Ela'), ('Potro', 'Potranca'), ('Pais', 'Mães'), ('esposas', 'maridos'), ('netos', 'netas'), ('MENINO', 'MENINA'), ('sobrinho', 'sobrinha'), ('VEREADOR', 'VEREADORA'), ('PATERNIDADE', 'MATERNIDADE'), ('Neto', 'Neta'), ('filho', 'filha'), ('REIS', 'RAINHAS'), ('HOMENS', 'MULHERES'), ('ESPOSAS', 'MARIDOS'), ('MONAST

0.015151515151515152